## **rebuilding simplified self-attention from scratch**

steps
1. compute the attention score for a query(i other words a token/word), journey, then scale is up to all
2. normalize attentiona scores vector by applying softmax
3. multiply attention weight my inputs to get the context vector

In [1]:
import torch

In [ ]:
# computing context vector for a query(simple a word)
# Corresponding words
words = ['Your', 'journey', 'starts', 'with', 'one', 'step']

inputs = torch.tensor(
  [
   [0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55], # step     (x^6)
  ]
)

journey_query = inputs[1] # which is journey
attn_scores_journey = torch.zeros(inputs.shape[0])

tensor([0., 0., 0., 0., 0., 0.])

In [10]:
# computing attention score for the query journey
# so the attention score for journey query is computed by doing a dot product journey query against all the other query in the input token or sequence

for i, x_i in enumerate(inputs):
    # so here we do dot product of each query again the journey to get the attention scores vector
    attn_scores_journey[i] = torch.dot(x_i, journey_query)
    

In [26]:
attn_scores_journey

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])

In [17]:
# attn_scores_journey.size()
# print("journey attention scores are", attn_scores_journey)

# now we normalize the attention score
attn_scores_journey_weights = torch.softmax(attn_scores_journey, dim=0)
attn_scores_journey_weights

tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])

In [22]:
# we now compute the context vector
# this well be done by first multiplyin the attention weight by each input embeddings and then summing them all up
journey_query_context_vector = torch.zeros(inputs.shape[-1])

for i, x_i in enumerate(inputs):
   journey_query_context_vector += attn_scores_journey_weights[i] * x_i


print("journey's context vector is ", journey_query_context_vector)

journey's context vector is  tensor([0.4419, 0.6515, 0.5683])


**now computing context vecore for all input embedding... scaling it up**
![](../../images/L14_s7.png)

### two main rules of matrix mul
1. the inner dimention must always much
    - `(3, 2) @ (3, 2)` won't work ❌❌
    - `(3, 2) @ (2, 3)` will work ✅✅
    - `(2, 3) @ (3, 2)` will work ✅✅

2. the resulting matrix has always has the shape fo the outer dimension
    - `(2,4) @ (4, 2)` -> `(2, 2)`
    - `(5, 3) @ (3, 4)` -> `(5, 4)`

In [27]:
inputs.shape, inputs.T.shape

(torch.Size([6, 3]), torch.Size([3, 6]))

In [28]:
inputs, inputs.T

(tensor([[0.4300, 0.1500, 0.8900],
         [0.5500, 0.8700, 0.6600],
         [0.5700, 0.8500, 0.6400],
         [0.2200, 0.5800, 0.3300],
         [0.7700, 0.2500, 0.1000],
         [0.0500, 0.8000, 0.5500]]),
 tensor([[0.4300, 0.5500, 0.5700, 0.2200, 0.7700, 0.0500],
         [0.1500, 0.8700, 0.8500, 0.5800, 0.2500, 0.8000],
         [0.8900, 0.6600, 0.6400, 0.3300, 0.1000, 0.5500]]))

In [29]:
# computing attention scores for all input embedding using matrix multiplication
attn_matrix = inputs @ inputs.T # this also works -> torch.matmul(inputs, inputs.T)
attn_matrix

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])

In [40]:
attn_weight_matrix = torch.softmax(attn_matrix, dim=-1)
attn_weight_matrix

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])

In [41]:
attn_matrix.shape, inputs.shape

(torch.Size([6, 6]), torch.Size([6, 3]))

In [42]:
# now that we have our attention weights, we compute the context vector
context_vector = attn_weight_matrix @ inputs
context_vector

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])

In [44]:
# checking to see if the individual computation i did was similar to doing all the computation at a go using the matrix multiplication.. and they are actually quite similar
journey_query_context_vector, context_vector[1]

(tensor([0.4419, 0.6515, 0.5683]), tensor([0.4419, 0.6515, 0.5683]))